In [1]:
from binance import Client
import personal_function as pf
import pandas as pd
import numpy as np
import mplfinance as mpf
from binance.enums import *
#from __future__ import (absolute_import, division, print_function,
#                        unicode_literals)

import datetime  # For datetime objects
#import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
#import argparse
# Import the backtrader platform
import backtrader as bt

In [2]:
client=pf.client()

KlinesType ='SPOT'
symbol ='BTCUSDT'
interval = '4h' # 30m, 4h,8h,12h,3d,1w
start_date ='1 Jan 2011'
end_date = '1 Jan 2021'
#limit = 500
hist_df=pf.historical_data(client,symbol, interval, start_date, end_date, KlinesType)

In [3]:
hist_df.head()
hist_df.drop(columns=['Quote Asset Volume', 'Number of Trades', 'TB Base Volume', 'TB Quote Volume', 'Ignore','Close Time'],inplace=True)
#hist_df.drop(columns=['Close Time'], inplace=True)
hist_df.dtypes
hist_df.head()
hist_df.to_csv('{}.csv'.format(symbol))
hist_df
#first_column = hist_df.iloc[:, 0]
#first_column


,Open Time,Open,High,Low,Close,Volume
0,2017-08-17 04:00:00,4261.48,4349.99,4261.32,4349.99,82.088865
1,2017-08-17 08:00:00,4333.32,4485.39,4333.32,4427.30,63.619882
2,2017-08-17 12:00:00,4436.06,4485.39,4333.42,4352.34,174.562001
3,2017-08-17 16:00:00,4352.33,4354.84,4200.74,4325.23,225.109716
4,2017-08-17 20:00:00,4307.56,4369.69,4258.56,4285.08,249.769913
...,...,...,...,...,...,...
7376,2020-12-31 08:00:00,29155.24,29210.83,28580.00,28910.30,10508.482930
7377,2020-12-31 12:00:00,28910.29,28989.03,27850.00,28770.00,19597.147389
7378,2020-12-31 16:00:00,28782.01,29000.00,28311.00,28897.83,10279.179141
7379,2020-12-31 20:00:00,28897.84,29169.55,28780.00,28923.63,7875.879035


In [4]:
training_df=hist_df[0:int((hist_df.shape[0]/2))+1]
training_df.to_csv('{}_training.csv'.format(symbol))
test_df=hist_df[int((hist_df.shape[0]/2))+1:]
test_df.to_csv('{}_test.csv'.format(symbol))


In [5]:
test_df

,Open Time,Open,High,Low,Close,Volume
3691,2019-04-26 12:00:00,5333.30,5339.55,5274.00,5285.07,6197.502081
3692,2019-04-26 16:00:00,5285.17,5288.37,5195.89,5260.61,7390.853002
3693,2019-04-26 20:00:00,5260.46,5340.00,5257.48,5314.10,4169.456561
3694,2019-04-27 00:00:00,5315.00,5326.12,5287.00,5288.27,2407.676701
3695,2019-04-27 04:00:00,5287.50,5325.59,5261.29,5323.00,2901.135414
...,...,...,...,...,...,...
7376,2020-12-31 08:00:00,29155.24,29210.83,28580.00,28910.30,10508.482930
7377,2020-12-31 12:00:00,28910.29,28989.03,27850.00,28770.00,19597.147389
7378,2020-12-31 16:00:00,28782.01,29000.00,28311.00,28897.83,10279.179141
7379,2020-12-31 20:00:00,28897.84,29169.55,28780.00,28923.63,7875.879035


In [6]:
class TestStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.datetime(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].low
    
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

data=bt.feeds.GenericCSVData(
    dataname='{}_training.csv'.format(symbol),
    fromdate=datetime.datetime(2017,8,17,4,0,0),
    todate=datetime.datetime(2019,4,26,8,0,0),
    datetime=1,
    open=2,
    high=3,
    low=4,
    close=5,
    volume=6,
    dtformat=('%Y-%m-%d %H:%M:%S'),
    tmformat=('%H:%M:%S'),
    timeframe=bt.feeds.TimeFrame.Ticks
)


cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(100000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2017-08-17T04:00:00, Close, 4261.32
2017-08-17T08:00:00, Close, 4333.32
2017-08-17T12:00:00, Close, 4333.42
2017-08-17T16:00:00, Close, 4200.74
2017-08-17T20:00:00, Close, 4258.56
2017-08-18T00:00:00, Close, 4134.61
2017-08-18T04:00:00, Close, 4234.43
2017-08-18T08:00:00, Close, 4259.85
2017-08-18T12:00:00, Close, 4193.70
2017-08-18T16:00:00, Close, 4015.70
2017-08-18T20:00:00, Close, 3938.77
2017-08-19T00:00:00, Close, 4084.28
2017-08-19T04:00:00, Close, 3933.21
2017-08-19T08:00:00, Close, 3850.00
2017-08-19T12:00:00, Close, 3928.89
2017-08-19T16:00:00, Close, 3953.40
2017-08-19T20:00:00, Close, 4051.47
2017-08-20T00:00:00, Close, 4044.00
2017-08-20T04:00:00, Close, 4081.19
2017-08-20T08:00:00, Close, 4032.62
2017-08-20T12:00:00, Close, 4063.35
2017-08-20T16:00:00, Close, 4091.59
2017-08-20T20:00:00, Close, 4069.13
2017-08-21T00:00:00, Close, 4039.00
2017-08-21T04:00:00, Close, 3953.40
2017-08-21T08:00:00, Close, 4000.01
2017-08-21T12:00:00, Close, 